In [4]:
from os import ST_NOSUID
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [ ]:
data_fake = pd.read_csv('Fake.csv')
data_true = pd.read_csv('True.csv')

In [ ]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
data_fake["class"]=0
data_true["class"]=1

In [ ]:
data_fake.shape,data_true.shape

((23481, 5), (21417, 5))

In [ ]:
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480,23470,-1):
  data_fake.drop([i], axis=0, inplace =True)

data_true_manual_testing = data_true.tail(10)
for i in range(21416,21406,-1):
  data_true.drop([i], axis=0, inplace =True)



In [ ]:
data_fake.shape,data_true.shape

((23471, 5), (21407, 5))

In [ ]:
data_fake_manual_testing["class"] =0
data_true_manual_testing["class"] =1

<ipython-input-9-fec87138bcb4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing["class"] =0
<ipython-input-9-fec87138bcb4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing["class"] =1


In [ ]:
data_fake_manual_testing.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [ ]:
data_true_manual_testing.head(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [ ]:
data_merge = pd.concat([data_fake,data_true],axis=0)
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [ ]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [ ]:
data = data_merge.drop(['title','subject','date'],axis=1)

In [ ]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [ ]:
data =data.sample(frac=1)

In [ ]:
data.head()

,text,class
10404,Nancy Pelosi had a strange reaction to a repor...,0
7979,WASHINGTON (Reuters) - The chairman of the U.S...,1
6566,"PALM BEACH, Fla. (Reuters) - U.S. President-el...",1
14468,I feel like I ve been drugged by Obama s rhet...,0
19313,Brigitte Gabriel was born in the Marjeyoun Dis...,0


In [ ]:
data.reset_index(inplace =True)
data.drop(['index'],axis=1,inplace=True)

In [ ]:
data.columns

Index(['text', 'class'], dtype='object')

In [ ]:
data.head()

,text,class
0,Nancy Pelosi had a strange reaction to a repor...,0
1,WASHINGTON (Reuters) - The chairman of the U.S...,1
2,"PALM BEACH, Fla. (Reuters) - U.S. President-el...",1
3,I feel like I ve been drugged by Obama s rhet...,0
4,Brigitte Gabriel was born in the Marjeyoun Dis...,0


In [ ]:
def wordopt(text):
  text = text.lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub("\\W", " ", text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' %re.escape(string.punctuation), '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)
  return text

In [ ]:
data['text'] = data['text'].apply(wordopt)

In [ ]:
x=data['text']
y=data['class']

In [ ]:
x_train,x_test,y_train,y_test =  train_test_split(x,y,test_size=0.25)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train =vectorization.fit_transform(x_train)
xv_test  = vectorization.transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [ ]:
pred_lr=LR.predict(xv_test)

In [ ]:
LR.score(xv_test,y_test)

0.9859180035650624

In [ ]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5940
           1       0.98      0.99      0.99      5280

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [ ]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [ ]:
pred_dt = DT.predict(xv_test)

In [ ]:
DT.score(xv_test, y_test)

0.9968805704099821

In [ ]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5940
           1       1.00      1.00      1.00      5280

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

GB=GradientBoostingClassifier(random_state =0)
GB.fit(xv_train, y_train)


GradientBoostingClassifier(random_state=0)

In [ ]:
pred_gb=GB.predict(xv_test)

In [ ]:
GB.score(xv_test, y_test)

0.9959001782531194

In [ ]:
print(classification_report(y_test, pred_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5940
           1       0.99      1.00      1.00      5280

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF= RandomForestClassifier(random_state=0)
RF.fit(xv_train,y_train)

RandomForestClassifier(random_state=0)

In [ ]:
pred_rf=RF.predict(xv_test)

In [ ]:
RF.score(xv_test,y_test)

0.9906417112299465

In [ ]:
print(classification_report(y_test,pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5940
           1       0.99      0.99      0.99      5280

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [2]:
def output_lable(n):
  if n==0:
    return "Fake News"
  elif n == 1:
    return "Not A Fake News"

def manual_testing(news):
  testing_news = {"text": [news]}
  new_def_test = pd.DataFrame(testing_news)
  new_def_test["text"] = new_def_test["text"].apply(wordopt)
  new_x_test = new_def_test["text"]
  new_xv_test = vectorization.transform(new_x_test)
  pred_LR = LR.predict(new_xv_test)
  pred_DT = DT.predict(new_xv_test)
  pred_GB = GB.predict(new_xv_test)
  pred_RF = RF.predict(new_xv_test)

  return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGB Prediction: {} \nRF Prediction: {}".format(output_lable(pred_LR[0]),
                                                                                                          output_lable(pred_DT[0]),
                                                                                                          output_lable(pred_GB[0]),
                                                                                                          output_lable(pred_RF[0])))

In [ ]:
news=str(input())
manual_testing(news)

Young voters in key election battlegrounds are being recommended fake AI-generated videos featuring party leaders, misinformation, and clips littered with abusive comments, the BBC has found. With TikTok emerging as a new social media battleground in this election, the political parties have begun a war of memes on the app in a bid to reach its audience of young voters. But a BBC project to investigate the content promoted by social media algorithms has found - alongside funny montages - young people on TikTok are being exposed to misleading and divisive content. It is being shared by everyone from students and political activists to comedians and anonymous bot-like accounts.


LR Prediction: Fake News 
DT Prediction: Fake News 
GB Prediction: Fake News 
RF Prediction: Fake News


In [ ]:
news=str(input())
manual_testing(news)

Ethiopia is the most populous nation in East Africa. In the 20th century, it faced economic troubles and civil war between communist and non-communist forces. Today, it is quickly developing, building infrastructure and factories. Much of the capital for this development comes from foreign partners, especially China, whose ambitious Belt and Road Initiative has focused on this core of the Horn of Africa. Ethiopia’s industrial sector now employs millions of people, and Ethiopians are growing more prosperous. Yet it is not rosy for everyone; indeed, any development this large and fast will have uneven effects on the human lives involved in it.


LR Prediction: Fake News 
DT Prediction: Fake News 
GB Prediction: Fake News 
RF Prediction: Fake News


In [ ]:
news=str(input())
manual_testing(news)

sun rise in the east


LR Prediction: Fake News 
DT Prediction: Fake News 
GB Prediction: Fake News 
RF Prediction: Fake News


In [ ]:
news=str(input())
manual_testing(news)

Andhra Pradesh Municipal Administration Minister P Narayana on Sunday said here that the Amaravati capital city development works will be completed in two-and-a-half years' time. Narayana said the state capital of Amaravati will be developed as one of the top five capital cities in the world and that the plans devised in the past will be implemented to meet this target.


LR Prediction: Not A Fake News 
DT Prediction: Fake News 
GB Prediction: Fake News 
RF Prediction: Not A Fake News


In [ ]:
news=str(input())
manual_testing(news)



LR Prediction: Fake News 
DT Prediction: Fake News 
GB Prediction: Fake News 
RF Prediction: Fake News


In [ ]:
news=str(input())
manual_testing(news)

The number of cases of cops brutalizing and killing people of color seems to see no end. Now, we have another case that needs to be shared far and wide. An Alabama woman by the name of Angela Williams shared a graphic photo of her son, lying in a hospital bed with a beaten and fractured face, on Facebook. It needs to be shared far and wide, because this is unacceptable.It is unclear why Williams  son was in police custody or what sort of altercation resulted in his arrest, but when you see the photo you will realize that these details matter not. Cops are not supposed to beat and brutalize those in their custody. In the post you are about to see, Ms. Williams expresses her hope that the cops had their body cameras on while they were beating her son, but I think we all know that there will be some kind of convenient  malfunction  to explain away the lack of existence of dash or body camera footage of what was clearly a brutal beating. Hell, it could even be described as attempted murder

In [ ]:
news=str(input())
manual_testing(news)

(Reuters) - Alabama officials on Thursday certified Democrat Doug Jones the winner of the stateâ€™s U.S. Senate race, after a state judge denied a challenge by Republican Roy Moore, whose campaign was derailed by accusations of sexual misconduct with teenage girls. Jones won the vacant seat by about 22,000 votes, or 1.6 percentage points, election officials said. That made him the first Democrat in a quarter of a century to win a Senate seat in Alabama.  The seat was previously held by Republican Jeff Sessions, who was tapped by U.S. President Donald Trump as attorney general. A state canvassing board composed of Alabama Secretary of State John Merrill, Governor Kay Ivey and Attorney General Steve Marshall certified the election results. Seating Jones will narrow the Republican majority in the Senate to 51 of 100 seats. In a statement, Jones called his victory â€œa new chapterâ€ and pledged to work with both parties. Moore declined to concede defeat even after Trump urged him to do so

In [ ]:
news=str(input())
manual_testing(news)

YSR Congress Party (YSRCP) president Y.S. Jagan Mohan Reddy has asserted that his party will extend issue based support keeping the State and nation’s interests in mind. In the same breath, he reminded the National Democratic Alliance (NDA) that his party has 15 MPs in Parliament. The former Chief Minister was interacting with the YSRCP MPs at his camp office at Tadepalli, near here, on Friday.The YSRCP had 11 MPs in Rajya Sabha, and four MPs in Lok Sabha taking the total strength to 15. The Telugu Desam Party (TDP), on other hand, had 16 MPs, he said, adding, “So, the YSRCP is equally powerful, none can touch us. We need to be brave and stand by the people.”  The YSRCP MPs would have to take the nation’s interest into consideration when they were in Parliament. The YSRCP  remained open to extending issue-based support to the BJP-led NDA government if the matter was in the interest of the country and Andhra Pradesh. Vijay Sai Reddy would continue to be YSRCP leader in Rajya Sabha, and 

In [ ]:
news=str(input())
manual_testing(news)

YSR Congress Party (YSRCP) president Y.S. Jagan Mohan Reddy has asserted that his party will extend issue based support keeping the State and nation’s interests in mind. In the same breath, he reminded the National Democratic Alliance (NDA) that his party has 15 MPs in Parliament. The former Chief Minister was interacting with the YSRCP MPs at his camp office at Tadepalli, near here, on Friday.The YSRCP had 11 MPs in Rajya Sabha, and four MPs in Lok Sabha taking the total strength to 15. The Telugu Desam Party (TDP), on other hand, had 16 MPs, he said, adding, “So, the YSRCP is equally powerful, none can touch us. We need to be brave and stand by the people.”  The YSRCP MPs would have to take the nation’s interest into consideration when they were in Parliament. The YSRCP  remained open to extending issue-based support to the BJP-led NDA government if the matter was in the interest of the country and Andhra Pradesh. Vijay Sai Reddy would continue to be YSRCP leader in Rajya Sabha, and 

In [ ]:
news=str(input())
manual_testing(news)

By now, the whole world knows that Alabama Senate candidate Roy Moore (R-Of Course) was banned from an Alabama mall and the YMCA for creeping on little girls. He has been accused of molesting young girls as young as age fourteen. Of course, when the allegations first came out and the uproar and backlash began, everyone, regardless of politics, reacted with outrage. However, the GOP s outrage was often much more muted. Further, it took awhile for the Republican National Committee to pull their support for Moore, despite the deeply disturbing allegations of his being a pedophile. Well, now that the rage has died down, the RNC is back with a new message regarding this Senate race: Better a pedophile than a Democrat.Under cover of night, the RNC reinstated their support for Roy Moore, and an RNC official confirmed to The Hill that, quote, We can confirm our involvement in the Alabama Senate race. So, there you have it, folks. They literally want a child molester in the United States Senate

In [ ]:
news=str(input())
manual_testing(news)

News Decoder started with one journalist who sought to build global awareness in young people. Over the next nine years dozens of other journalists signed on.When News Decoder got its start it was with the idea of helping young people understand how they are connected to global events and things happening far from where they live.  And even as we have expanded, working with schools to teach journalism, connecting students across borders through Zoom dialogues, and inspiring students to create podcasts through News Decoder clubs and camps, we’ve stayed faithful to that mission.  We now work with dozens of correspondents in countries all over the word: Peru, Hungary, Canada, Switzerland, Uganda, India, Sri Lanka, Rwanda, Australia, Indonesia, to name just a few places.  They write about complex topics: development funding, immigration, environmental conservation, international treaties, human rights, healthcare and war in a way that can engage students and help them understand important 

In [ ]:
news=str(input())
manual_testing(news)

Donald Trump has a white supremacy problem, and now it is causing Great Britain, America s closest ally, to rebuke him in an historically public way. After Trump retweeted the leader of Britain First, a fascist white nationalist group, Prime Minister Theresa May decided she had had enough. She publicly criticized Trump, and made it clear that she in no way approved of his identifying with Britain First. May said of Trump s tweets: The fact that we work together does not mean that we re afraid to say when we think the United States has got it wrong, and be very clear with them. And I m very clear that retweeting from Britain First was the wrong thing to do. May went on to call Britain First a  hateful organization  that has caused division in her nation. Now, it seems that a so-called  working visit  that had been planned soon for Trump in Britain will now be cancelled.Trump, of course, instead of trying to patch things up in the  special relationship  that has always existed between th

In [3]:
news=str(input())
manual_testing(news)

sun rises in the west and sets in the east


NameError: name 'pd' is not defined